In [2]:
import pandas as pd
import numpy as np

## Filtrar equipos de interés

In [2]:
equipos_24 = ['eng Arsenal', 'eng Manchester City', 
              'es Atlético Madrid', 'es Barcelona', 'es Real Madrid', 'es Real Sociedad',
              'de Bayern Munich', 'de Dortmund', 'de RB Leipzig',
              'it Inter', 'it Lazio', 'it Napoli',
              'fr Paris S-G',
              'pt Porto',
              'nl PSV Eindhoven',
              'dk FC Copenhagen' ]
print(len(equipos_24))

categ_estad = ['standard', 'goalkeeping', 'shooting', 'goal and shot creation']

16


In [22]:
# pasar los csv a dataframe
dict_categ_eq_23 = {}
dict_categ_eq_24 = {}
dict_categ_jug_23 = {}
dict_categ_jug_24 = {}


for anio in [2023, 2024]:
    for categ in categ_estad:
        df_eq = pd.read_csv(f'data/UCL_{anio}/est_equipos/{categ}.csv')
        df_jug = pd.read_csv(f'data/UCL_{anio}/est_jugadores/{categ}.csv')

        if anio == 2023:
            dict_categ_eq_23[categ] = df_eq
            dict_categ_jug_23[categ] = df_jug
        else:
            dict_categ_eq_24[categ] = df_eq
            dict_categ_jug_24[categ] = df_jug

In [23]:
# filtar los dataframes solo los equipos que nos interesan
dict_categ_eq_23 = {categ: df[df['Squad'].isin(equipos_24)] for categ, df in dict_categ_eq_23.items()}
print(dict_categ_eq_23['standard'].shape)
dict_categ_eq_24 = {categ: df[df['Squad'].isin(equipos_24)] for categ, df in dict_categ_eq_24.items()}
print(dict_categ_eq_24['standard'].shape)
print()
dict_categ_jug_23 = {categ: df[df['Squad'].isin(equipos_24)] for categ, df in dict_categ_jug_23.items()}
print(dict_categ_jug_23['standard'].shape)
dict_categ_jug_24 = {categ: df[df['Squad'].isin(equipos_24)] for categ, df in dict_categ_jug_24.items()}
print(dict_categ_jug_24['standard'].shape)

(12, 33)
(16, 33)

(279, 39)
(366, 39)


In [24]:
dict_categ_eq_24['standard'].head(3)

,Team ID,Squad,# Pl,Age,Poss,MP,Starts,Min,90s,Gls,...,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,Per 90 Minutes
1,18bb7c10,eng Arsenal,22,25.8,57.0,7,77,630,7.0,16,...,2.29,1.86,4.14,2.14,4.00,1.59,1.24,2.83,1.48,2.72
2,db3b9613,es Atlético Madrid,22,29.9,51.9,7,77,630,7.0,15,...,2.14,1.29,3.43,2.14,3.43,1.41,1.05,2.46,1.28,2.32
3,206d90db,es Barcelona,24,27.1,60.0,7,77,630,7.0,12,...,1.71,1.71,3.43,1.71,3.43,1.60,1.38,2.98,1.60,2.98


In [25]:
dict_categ_jug_24['standard'].head(3)

,Player ID,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,...,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches,Player Link
0,5f09991f,1,Patrick van Aanholt,nl NED,"MF,DF",nl PSV Eindhoven,33-193,1990,2,1,...,0.00,0.0,0.00,0.15,0.03,0.18,0.15,0.18,Matches,https://fbref.com/en/players/5f09991f/Patrick-...
3,b96b595c,4,Francesco Acerbi,it ITA,DF,it Inter,36-028,1988,5,4,...,0.24,0.0,0.24,0.05,0.07,0.12,0.05,0.12,Matches,https://fbref.com/en/players/b96b595c/Francesc...
4,4dd2a0be,5,Elias Achouri,tn TUN,"FW,MF",dk FC Copenhagen,25-028,1999,8,7,...,0.16,0.0,0.16,0.05,0.14,0.19,0.05,0.19,Matches,https://fbref.com/en/players/4dd2a0be/Elias-Ac...


In [26]:
lista_df_eq = [dict_categ_eq_23, dict_categ_eq_24]
lista_df_jug = [dict_categ_jug_23, dict_categ_jug_24]

## Limpieza nulos y duplicados

In [21]:
for dict in lista_df_jug:
    for categ, df in dict.items():
        print(df['Player ID'].value_counts().loc[lambda x: x>1])


Player ID
bd6351cd    2
Name: count, dtype: int64
Series([], Name: count, dtype: int64)
Player ID
bd6351cd    2
Name: count, dtype: int64
Player ID
bd6351cd    2
Name: count, dtype: int64
Player ID
cf007308    2
a4b00319    2
Name: count, dtype: int64
Series([], Name: count, dtype: int64)
Player ID
cf007308    2
a4b00319    2
Name: count, dtype: int64
Player ID
cf007308    2
a4b00319    2
Name: count, dtype: int64


Hay 3 jugadores duplicados por traspasos/cesiones

In [ ]:
jugadores_dupl = ['bd6351cd', 'cf007308', 'a4b00319']

Eliminar filas con valores nulos

## Variables de interés en cada categoría

In [8]:
# vamos a sacar las variables (nombre columnas) de cada categoría
dict_var_eq = {}
dict_var_jug = {}
    # KEYS: categorías de estadisticas
    # VALUES: nombres de las columnas de las estadisticas

for categ in categ_estad:
    dict_var_eq[categ] = dict_categ_eq_23[categ].columns.tolist()
    dict_var_jug[categ] = dict_categ_jug_23[categ].columns.tolist()

### VARIABLES '*standard*' DE INTERÉS  
De equipos y jugadores:
- ***'Gls.1'***: goles por partido
- ***''Ast.1'***: asistencias por partido
- ***''G+A.1'***: goles + asistencias por partido
- ***''G-PK.1'***: goles (sin penaltis) por partido
- ***''G+A-PK'***: goles + asistencias (sin penaltis) por partido

In [9]:
print(dict_var_eq['standard'])
print(dict_var_jug['standard'])

['Team ID', 'Squad', '# Pl', 'Age', 'Poss', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'Per 90 Minutes']
['Player ID', 'Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1', 'xG+xAG', 'npxG.1', 'npxG+xAG.1', 'Matches', 'Player Link']


### VARIABLES '*goal and shot creation*' DE INTERÉS
- ***'SCA90'***: acciones que acaban en tiro (por partido)
- ***'GCA90'***: acciones que acaban en gol (por partido)


In [11]:
print(dict_var_eq['goal and shot creation'])
print(dict_var_jug['goal and shot creation'])

['Team ID', 'Squad', '# Pl', '90s', 'SCA', 'SCA90', 'PassLive', 'PassDead', 'TO', 'Sh', 'Fld', 'Def', 'GCA', 'GCA90', 'PassLive.1', 'PassDead.1', 'TO.1', 'Sh.1', 'Fld.1', 'GCA Types']
['Player ID', 'Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'SCA', 'SCA90', 'PassLive', 'PassDead', 'TO', 'Sh', 'Fld', 'Def', 'GCA', 'GCA90', 'PassLive.1', 'PassDead.1', 'TO.1', 'Sh.1', 'Fld.1', 'Def.1', 'Matches', 'Player Link']


### VARIABLES '*shooting*' DE INTERÉS
- **'Sh/90'**: tiros por partido
- **'SoT/90'**: tiros a puerta por partido
- **'SoT%'**: % de tiros a puerta
- **'G/Sh'**: goles por tiro
- **'G/SoT'**: goles por tiro a puerta



In [12]:
print(dict_var_eq['shooting'])
print(dict_var_jug['shooting'])

['Team ID', 'Squad', '# Pl', '90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'Expected']
['Player ID', 'Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG', 'Matches', 'Player Link']


### VARIABLES '*goalkeeping*' DE INTERÉS
- **'GA90'**: goles recibidos por partido
- **'Save%'**: % de salvadas
- **'SoTA'**: tiros a puerta recibidos por partido (alto valor = mala defensa)




In [13]:
print(dict_var_eq['goalkeeping'])
print(dict_var_jug['goalkeeping'])

['Team ID', 'Squad', '# Pl', 'MP', 'Starts', 'Min', '90s', 'GA', 'GA90', 'SoTA', 'Saves', 'Save%', 'W', 'D', 'L', 'CS', 'CS%', 'PKatt', 'PKA', 'PKsv', 'PKm', 'Penalty Kicks']
['Player ID', 'Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'GA', 'GA90', 'SoTA', 'Saves', 'Save%', 'W', 'D', 'L', 'CS', 'CS%', 'PKatt', 'PKA', 'PKsv', 'PKm', 'Save%.1', 'Matches', 'Player Link']


## Asignar variables

In [14]:
var_general = ['Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK']

# ataque
var_tiro = ['Sh/90', 'SoT/90', 'SoT%', 'G/Sh', 'G/SoT']
var_gol = ['SCA90', 'GCA90']

# defensa
var_portero = ['GA90', 'Save%', 'SoTA']

In [20]:
# para cada categoria, crear dataframes de solo las variables de interes
def func_asignar_categ_a_var(categ):
    if categ=='standard':
        return var_general
    elif categ=='goal and shot creation':
        return var_gol
    elif categ=='shooting':
        return var_tiro
    elif categ=='goalkeeping':
        return var_portero


def reducir_df_a_var_interes(anio, dict_categ, is_equipos:bool):
    if is_equipos:
        for categ, df in dict_categ.items():
            var_eqipo = ['Team ID', 'Squad'] + func_asignar_categ_a_var(categ)
            df_final = df[var_eqipo]
            df_final.set_index('Team ID', inplace=True)
            df_final.to_csv(f'data/UCL_{anio}/EQUIPO_{categ}.csv')

    else: # si es jugadores
        for categ, df in dict_categ.items():
            var_jug = ['Player ID', 'Player', 'Pos','Squad'] + func_asignar_categ_a_var(categ)
            df_final = df[var_jug]
            df_final.set_index('Player ID', inplace=True)
            df_final.to_csv(f'data/UCL_{anio}/JUGADOR_{categ}.csv')


        

In [28]:
reducir_df_a_var_interes(2023, dict_categ_eq_23, True)
reducir_df_a_var_interes(2024,dict_categ_eq_24, True)
reducir_df_a_var_interes(2023,dict_categ_jug_23, False)
reducir_df_a_var_interes(2024,dict_categ_jug_24, False)

## Media jugadores

In [3]:
df_eq_23 = pd.read_csv('data/EQUIPO_2023.csv')
df_jug_23 = pd.read_csv('data/JUGADOR_2023.csv')

In [4]:
madrid = df_jug_23[df_jug_23['Squad']=='es Real Madrid']
madrid.head(3)

,Player ID,Player,Pos,Squad,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,Sh/90,SoT/90,SoT%,G/Sh,G/SoT,SCA90,GCA90,GA90,Save%,SoTA
4,05439de2,David Alaba,DF,es Real Madrid,0.00,0.00,0.00,0.00,0.00,0.63,0.38,60.0,0.00,0.00,1.64,0.13,NaN,NaN,NaN
11,45af8a54,Marco Asensio,"FW,MF",es Real Madrid,0.83,0.28,1.11,0.83,1.11,3.33,1.39,41.7,0.25,0.60,4.72,0.83,NaN,NaN,NaN
21,70d74ece,Karim Benzema,FW,es Real Madrid,0.48,0.12,0.60,0.48,0.60,4.23,2.18,51.4,0.11,0.22,4.35,0.48,NaN,NaN,NaN


In [6]:
madrid.shape

(22, 19)

In [9]:
madrid_gol = madrid[madrid['Gls.1']>0]
print(madrid_gol.shape)
print( madrid_gol['Gls.1'].mean() )

(9, 19)
0.43
